# Загрузка библиотек

In [483]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
df = pd.read_csv('dogs-ranking-dataset.csv')

# Предобработка данных

In [484]:
df

,Breed,type,score,popularity ranking,size,intelligence,congential ailments,score for kids,size.1,$LIFETIME COST,INTELLIGENCE RANK,INTELLIGENCE %,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,Border Terrier,terrier,3.61,61,1,Above average,none,4.99,small,"$22,638",30,70%,14.00,0,none,$833,$324,Once a week,1
1,Cairn Terrier,terrier,3.53,48,1,Above average,"'lion jaw', heart problems",4.91,small,"$21,992",35,61%,13.84,2,"'lion jaw', heart problems",$435,$324,Once a week,1
2,Siberian Husky,working,3.22,16,2,Average,none,4.72,medium,"$22,049",45,45%,12.58,0,none,$650,$466,Once in a few weeks,1
3,Welsh Springer Spaniel,sporting,3.34,81,2,Above average,hip problems,4.71,medium,"$20,224",31,69%,12.49,1,hip problems,$750,$324,Once a week,1
4,English Cocker Spaniel,sporting,3.33,51,2,Excellent,none,4.70,medium,"$18,993",18,82%,11.66,0,none,$800,$324,Once a week,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Alaskan Malamute,working,1.82,47,3,Average,"hip problems, dwarfism",2.57,large,"$21,986",50,36%,10.67,2,"hip problems, dwarfism","$1,210",$710,Daily,2
83,Bloodhound,hound,1.66,42,3,Lowest,"fatal stomach bloat, skin problems",2.54,large,"$13,824",75,7%,6.75,2,"fatal stomach bloat, skin problems",$608,$710,Once a week,2
84,Chow Chow,non-sporting,1.76,54,2,Lowest,"eye, hip problems",2.51,medium,"$15,898",77,5%,9.01,2,"eye, hip problems",$515,$466,Daily,2
85,Akita,working,1.95,41,3,Average,hip problems,2.33,large,"$20,994",54,31%,10.16,1,hip problems,"$1,202",$710,Once a week,3


Данный датасет представляет собой набор предикторов и субъективную оценку для каждой представленной породы собак. Он содержит следующие столбцы:
* Порода
* Тип породы
* Оценка
* Ранг популярности
* Размер в номинальном формате
* Интеллект
* Врожденные заболевания
* Оценка для детей
* Размер в строковом формате
* Стоимость за жизнь
* Ранг интеллекта
* Процент интеллекта
* Продолжительность жизни
* Количество генетических заболеваний
* Генетические заболевания
* Цена покупки
* Стоимость корма за год
* Частота ухода за шерстью
* Насколько порода подходит для детей

Признаков очень много и они очень разношерстны. Некоторые повторяются, а некоторые, вероятно, вычислены исходя из других признаков в датасете.

Начнем причесывание данных, убрав название породы и ранг популярности за незначимостью.

In [485]:
df.drop(['Breed', 'popularity ranking'], axis=1, inplace=True)

 Далее возьмем все признаки, связанные с интеллектом, и оставим только один из них.

In [486]:
df[['INTELLIGENCE %', 'intelligence', 'INTELLIGENCE RANK']]

,INTELLIGENCE %,intelligence,INTELLIGENCE RANK
0,70%,Above average,30
1,61%,Above average,35
2,45%,Average,45
3,69%,Above average,31
4,82%,Excellent,18
...,...,...,...
82,36%,Average,50
83,7%,Lowest,75
84,5%,Lowest,77
85,31%,Average,54


Для анализа числовая переменная будет удобнее, чем категориальная или ранговая, поэтому преобразуем проценты в десятичную дробь и избавимся от остальных двух столбцов.

In [487]:
df[['INTELLIGENCE %']] = pd.DataFrame(df['INTELLIGENCE %'].str[:-1].astype('int') / 100)
df.drop(['intelligence', 'INTELLIGENCE RANK'], axis=1, inplace=True)
df.rename(columns={'INTELLIGENCE %': 'intelligence'}, inplace=True)
df.head(3)

,type,score,size,congential ailments,score for kids,size.1,$LIFETIME COST,intelligence,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,terrier,3.61,1,none,4.99,small,"$22,638",0.70,14.00,0,none,$833,$324,Once a week,1
1,terrier,3.53,1,"'lion jaw', heart problems",4.91,small,"$21,992",0.61,13.84,2,"'lion jaw', heart problems",$435,$324,Once a week,1
2,working,3.22,2,none,4.72,medium,"$22,049",0.45,12.58,0,none,$650,$466,Once in a few weeks,1


Два столбца связаны с размером, из них выберем ранговый.

In [488]:
df[['size', 'size.1']]

,size,size.1
0,1,small
1,1,small
2,2,medium
3,2,medium
4,2,medium
...,...,...
82,3,large
83,3,large
84,2,medium
85,3,large


In [489]:
df.drop('size.1', axis=1, inplace=True)
df.head(3)

,type,score,size,congential ailments,score for kids,$LIFETIME COST,intelligence,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,terrier,3.61,1,none,4.99,"$22,638",0.70,14.00,0,none,$833,$324,Once a week,1
1,terrier,3.53,1,"'lion jaw', heart problems",4.91,"$21,992",0.61,13.84,2,"'lion jaw', heart problems",$435,$324,Once a week,1
2,working,3.22,2,none,4.72,"$22,049",0.45,12.58,0,none,$650,$466,Once in a few weeks,1


Далее разберемся со здоровьем - с ним связано 3 столбца, 2 из которых, вероятно, совпадают, а третий содержит ту же информацию в более компактном виде.

In [490]:
df[['congential ailments', 'NUMBER OF GENETIC AILMENTS', 'GENETIC AILMENTS']]

,congential ailments,NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS
0,none,0,none
1,"'lion jaw', heart problems",2,"'lion jaw', heart problems"
2,none,0,none
3,hip problems,1,hip problems
4,none,0,none
...,...,...,...
82,"hip problems, dwarfism",2,"hip problems, dwarfism"
83,"fatal stomach bloat, skin problems",2,"fatal stomach bloat, skin problems"
84,"eye, hip problems",2,"eye, hip problems"
85,hip problems,1,hip problems


В этом случае мы так или иначе избавимся от строковых данных, но перед этим ради интереса сравним два столбца, которые выглядят одинаково.

In [491]:
df[df['congential ailments'] != df['GENETIC AILMENTS']][['congential ailments', 'NUMBER OF GENETIC AILMENTS', 'GENETIC AILMENTS']]

,congential ailments,NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS
6,'dry eye',1,dry eye
22,hip problems,1,no data
65,heart problems,1,heart problem
69,"heart, liver, hip problems",3,"heart, liver, hips"


Видно, что по своей сути данные совпадают везде. Они могли бы быть и содержательнее, например, включать серьезность каждой проблемы, но имеем, что имеем. Оставим только количество болезней.

In [492]:
df.drop(['congential ailments', 'GENETIC AILMENTS'], axis=1, inplace=True)
df.rename(columns={'NUMBER OF GENETIC AILMENTS': 'ailments'}, inplace=True)
df.head(3)

,type,score,size,score for kids,$LIFETIME COST,intelligence,LONGEVITY(YEARS),ailments,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,terrier,3.61,1,4.99,"$22,638",0.70,14.00,0,$833,$324,Once a week,1
1,terrier,3.53,1,4.91,"$21,992",0.61,13.84,2,$435,$324,Once a week,1
2,working,3.22,2,4.72,"$22,049",0.45,12.58,0,$650,$466,Once in a few weeks,1


Теперь посмотрим на цены. Есть подозрение, что lifetime cost вычислен как *purchase price + (food costs * longevity)*. Проверим данную гипотезу.

In [493]:
df[['$LIFETIME COST']] = pd.DataFrame(df['$LIFETIME COST'].str.replace('$', ''))
df[['$LIFETIME COST']] = pd.DataFrame(df['$LIFETIME COST'].str.replace(',', '').astype('int'))
df[['PURCHASE PRICE']] = pd.DataFrame(df['PURCHASE PRICE'].str.replace('$', ''))
df[['PURCHASE PRICE']] = pd.DataFrame(df['PURCHASE PRICE'].str.replace(',', '').astype('int'))
df[['FOOD COSTS PER YEAR']] = pd.DataFrame(df['FOOD COSTS PER YEAR'].str.replace('$', ''))
df[['FOOD COSTS PER YEAR']] = pd.DataFrame(df['FOOD COSTS PER YEAR'].str.replace(',', '').astype('int'))
df[['$LIFETIME COST', 'LONGEVITY(YEARS)', 'PURCHASE PRICE', 'FOOD COSTS PER YEAR']]

,$LIFETIME COST,LONGEVITY(YEARS),PURCHASE PRICE,FOOD COSTS PER YEAR
0,22638,14.00,833,324
1,21992,13.84,435,324
2,22049,12.58,650,466
3,20224,12.49,750,324
4,18993,11.66,800,324
...,...,...,...,...
82,21986,10.67,1210,710
83,13824,6.75,608,710
84,15898,9.01,515,466
85,20994,10.16,1202,710


In [494]:
df['$LIFETIME COST'] - (df['PURCHASE PRICE'] + df['FOOD COSTS PER YEAR'] * df['LONGEVITY(YEARS)'])

0     17269.00
1     17072.84
2     15536.72
3     15427.24
4     14415.16
        ...   
82    13200.30
83     8423.50
84    11184.34
85    12578.40
86     7867.86
Length: 87, dtype: float64

Видно, что на самом деле lifetime cost считается другим образом, скорее всего включая множество других факторов. Оставим для рассмотрения лишь этот столбец.

In [495]:
df.drop(['PURCHASE PRICE', 'FOOD COSTS PER YEAR'], axis=1, inplace=True)
df.rename(columns={'$LIFETIME COST': 'cost', 'LONGEVITY(YEARS)': 'longevity'}, inplace=True)
df.head(3)

,type,score,size,score for kids,cost,intelligence,longevity,ailments,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,terrier,3.61,1,4.99,22638,0.70,14.00,0,Once a week,1
1,terrier,3.53,1,4.91,21992,0.61,13.84,2,Once a week,1
2,working,3.22,2,4.72,22049,0.45,12.58,0,Once in a few weeks,1


С детьми также связаны два столбца, один числовой и один ранговый. Опять же, для исследования числовой будет более полезным.

In [496]:
df.drop('SUITABILITY FOR CHILDREN', axis=1, inplace=True)
df.rename(columns={'score for kids': 'kids_score'}, inplace=True)
df.head(3)

,type,score,size,kids_score,cost,intelligence,longevity,ailments,GROOMING FREQUNCY
0,terrier,3.61,1,4.99,22638,0.70,14.00,0,Once a week
1,terrier,3.53,1,4.91,21992,0.61,13.84,2,Once a week
2,working,3.22,2,4.72,22049,0.45,12.58,0,Once in a few weeks


Осталось посмотреть лишь на категориальные переменные. Тип породы будет полезен, поскольку также дает некоторые внешние и поведенческие характеристики, частота ухода за шерстью - удобство породы для содержания. Выведем уникальные значения обоих столбцов.

In [497]:
df[['GROOMING FREQUNCY']].value_counts()

GROOMING FREQUNCY  
Once a week            65
Daily                  20
Once in a few weeks     2
Name: count, dtype: int64

In [498]:
df.type.value_counts()

type
sporting        15
working         14
toy             13
hound           13
terrier         12
non-sporting    10
herding         10
Name: count, dtype: int64

Также, перед финальным преобразованием удостоверимся, что в данных нет пропусков.

In [499]:
df.isnull().sum()

type                 0
score                0
size                 0
kids_score           0
cost                 0
intelligence         0
longevity            0
ailments             0
GROOMING FREQUNCY    0
dtype: int64

В целом, к данным столбцам вопросов нет, используем one-hot encoding для преобразования их в более удобную форму.

In [500]:
df.rename(columns={'GROOMING FREQUNCY': 'grooming'}, inplace=True)
df_dummies = pd.get_dummies(df, drop_first=True).astype('float')
df_dummies

,score,size,kids_score,cost,intelligence,longevity,ailments,type_hound,type_non-sporting,type_sporting,type_terrier,type_toy,type_working,grooming_Once a week,grooming_Once in a few weeks
0,3.61,1.0,4.99,22638.0,0.70,14.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,3.53,1.0,4.91,21992.0,0.61,13.84,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,3.22,2.0,4.72,22049.0,0.45,12.58,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,3.34,2.0,4.71,20224.0,0.69,12.49,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,3.33,2.0,4.70,18993.0,0.82,11.66,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1.82,3.0,2.57,21986.0,0.36,10.67,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
83,1.66,3.0,2.54,13824.0,0.07,6.75,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
84,1.76,2.0,2.51,15898.0,0.05,9.01,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
85,1.95,3.0,2.33,20994.0,0.31,10.16,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


Датасет готов к дальнейшей работе.

# Создание и обучение модели

Для этих данных будем решать задачу регрессии, а именно предсказания субъективного score и kids_score согласно информации о породе. Таким образом, будем иметь две модели, и, возможно, разгадаем загадку, каким образом автор датасета оценивал собак.

In [501]:
from sklearn.linear_model import LinearRegression

Сперва используем регрессию для определения обычного score. Разобьем данные на тренировочные и тестовые и обучим модель. Оценим модель с помощью стандартной для регрессии метрики R-квадрат, ее предсказание - при помощи mean squared error.

In [502]:
x = df_dummies.drop(['score', 'kids_score'], axis=1)
y = df_dummies.score
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

lin = LinearRegression()
lin.fit(x_train, y_train)
print(lin.score(x_train, y_train))
mean_squared_error(lin.predict(x_test), y_test)

0.923770783789953


0.028431423951535927

Видно, что модель справилась со своей задачей очень хорошо - получился высокий R-квадрат и низкая среднеквадратическая ошибка. Теперь проведем подобную процедуру для kids_score.

In [503]:
x = df_dummies.drop(['score', 'kids_score'], axis=1)
y = df_dummies.kids_score
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

lin = LinearRegression()
lin.fit(x_train, y_train)
print(lin.score(x_train, y_train))
mean_squared_error(lin.predict(x_test), y_test)

0.7534434184526834


0.13818298772217183

Здесь точность оказалась гораздо ниже, и относительно высокой стала среднеквадратическая ошибка. Значит, линейная зависимость здесь не так сильна.

В итоге мы получили удачную линейную модель для предсказания обычной оценки породы на основе данных признаков и менее удачную для оценки породы как питомца для детей.